In [1]:
import functools
import inspect

In [2]:
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'function {fn.__name__} called {count} times')
        return fn(*args, **kwargs)
    return inner

def add(a, b):
    return a + b

In [3]:
add = counter(add)

In [4]:
add(10, 20)

function add called 1 times


30

In [5]:
add(3, 7)

function add called 2 times


10

# Decorator

In [6]:
def outer(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'function {fn.__name__} called {count} times')
        return fn(*args, **kwargs)
    return inner

@outer  # has same effect as : add = outer(add)
def add(a, b):
    return a + b

In [7]:
add(10,5)

function add called 1 times


15

In [8]:
print(f'Name: {add.__name__}\n')
print(f'Closure: {add.__closure__}\n')
print(f'Signature: {inspect.signature(add)}\n')
help(add)

Name: inner

Closure: (<cell at 0x107ad2ad0: int object at 0x10524a0c0>, <cell at 0x107ad2230: function object at 0x107ae5940>)

Signature: (*args, **kwargs)

Help on function inner in module __main__:

inner(*args, **kwargs)



### All the information about originall function `add` is lost and we get back details of `inner`. How can we fix this ?

In [10]:
def outer(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'function {fn.__name__} called {count} times')
        return fn(*args, **kwargs)
    inner.__name__ = fn.__name__
    inner.__doc__ = fn.__doc__
    return inner

@outer  # has same effect as : add = outer(add)
def add(a, b):
    """returns sum of a and b"""
    return a + b

In [11]:
print(f'Name: {add.__name__}\n')
print(f'Closure: {add.__closure__}\n')
print(f'Signature: {inspect.signature(add)}\n')
help(add)

Name: add

Closure: (<cell at 0x107aee9e0: int object at 0x10524a0a0>, <cell at 0x107aee980: function object at 0x107ae6160>)

Signature: (*args, **kwargs)

Help on function add in module __main__:

add(*args, **kwargs)
    returns sum of a and b



### But the function signature still reads `add(*args, **kwargs)`

In [12]:
def outer(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'function {fn.__name__} called {count} times')
        return fn(*args, **kwargs)
    inner = functools.wraps(fn)(inner)
    return inner

@outer  # has same effect as : add = outer(add)
def add(a, b):
    """returns sum of a and b"""
    return a + b

print(f'Name: {add.__name__}\n')
print(f'Closure: {add.__closure__}\n')
print(f'Signature: {inspect.signature(add)}\n')
help(add)

Name: add

Closure: (<cell at 0x107aeef20: int object at 0x10524a0a0>, <cell at 0x107aef730: function object at 0x107ae6ac0>)

Signature: (a, b)

Help on function add in module __main__:

add(a, b)
    returns sum of a and b



In [13]:
def outer(fn):
    count = 0
    @functools.wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'function {fn.__name__} called {count} times')
        return fn(*args, **kwargs)
    return inner

@outer  # has same effect as : add = outer(add)
def add(a, b):
    """returns sum of a and b"""
    return a + b

print(f'Name: {add.__name__}\n')
print(f'Closure: {add.__closure__}\n')
print(f'Signature: {inspect.signature(add)}\n')
help(add)

Name: add

Closure: (<cell at 0x107aefaf0: int object at 0x10524a0a0>, <cell at 0x107aec070: function object at 0x107ae6fc0>)

Signature: (a, b)

Help on function add in module __main__:

add(a, b)
    returns sum of a and b



# Application of Decorators

In [18]:
def timed(fn):
    # import these so users wont have to import these in their modules
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print(f'Elapsed time: {end - start}s')
        return result

    return inner

@timed
def add(a: int, b: int) -> int:
    return a + b
    

In [19]:
add(1, 7)

Elapsed time: 1.125037670135498e-06s


8

In [20]:
@timed
def fib(n):
    if n<=2:
        return 1
    else:
        return fib(n-1) + fib(n-1)

In [23]:
fib(5)

Elapsed time: 3.7497375160455704e-07s
Elapsed time: 3.33995558321476e-07s
Elapsed time: 0.00010679091792553663s
Elapsed time: 2.4994369596242905e-07s
Elapsed time: 2.50060111284256e-07s
Elapsed time: 3.0125025659799576e-05s
Elapsed time: 0.00016716693062335253s
Elapsed time: 2.50060111284256e-07s
Elapsed time: 2.909218892455101e-07s
Elapsed time: 3.0208961106836796e-05s
Elapsed time: 2.50060111284256e-07s
Elapsed time: 2.50060111284256e-07s
Elapsed time: 2.920802216976881e-05s
Elapsed time: 8.874991908669472e-05s
Elapsed time: 0.0002862090477719903s


8

## Whats happening ? 
Lets do some logging

In [37]:
def timed(fn):
    # import these so users wont have to import these in their modules
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        
        args_ = [str(x) for x in args]
        kwargs_ = ['{k}:{v}' for k,v in kwargs]
        
        print(f'{fn.__name__}({args_+kwargs_}) took: {end - start}s\n')
        return result

    return inner

@timed
def fib(n):
    if n<=2:
        return 1
    else:
        return fib(n-1) + fib(n-1)

In [38]:
fib(5)

fib(['2']) took: 5.830079317092896e-07s

fib(['2']) took: 6.25033862888813e-07s

fib(['3']) took: 0.00013025000225752592s

fib(['2']) took: 3.7497375160455704e-07s

fib(['2']) took: 3.7497375160455704e-07s

fib(['3']) took: 5.1332986913621426e-05s

fib(['4']) took: 0.00023408292327076197s

fib(['2']) took: 3.7497375160455704e-07s

fib(['2']) took: 4.580942913889885e-07s

fib(['3']) took: 4.904100205749273e-05s

fib(['2']) took: 3.7497375160455704e-07s

fib(['2']) took: 4.169996827840805e-07s

fib(['3']) took: 5.041609983891249e-05s

fib(['4']) took: 0.000148500083014369s

fib(['5']) took: 0.0004329170333221555s



8

# use lru cache, or reduce functions

In [40]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'{run_dt}: called {fn.__name__}')
        return result

    return inner

In [41]:
@logged
def func1():
    pass

In [42]:
@logged
def func2():
    pass

In [43]:
func1()

2025-06-21 17:44:46.496518+00:00: called func1


In [44]:
func2()

2025-06-21 17:44:49.523942+00:00: called func2


In [45]:
def timed(fn):
    from functools import wraps
    from time import perf_counter

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print(f'{fn.__name__} ran for {end-start}s')
        
        return result

    return inner

In [57]:
@logged
@timed
def fact(n):
    from functools import reduce
    return reduce(lambda x,y: x * y, range(1, n+1))

In [58]:
fact(5)

fact ran for 4.916917532682419e-06s
2025-06-21 17:51:35.088883+00:00: called fact


120

# above is same as 

In [61]:
def fact_2(n):
    from functools import reduce
    return reduce(lambda x,y: x * y, range(1, n+1))

fact_2 = logged(timed(fact_2))

In [62]:
fact_2(5)

fact_2 ran for 7.2499969974160194e-06s
2025-06-21 17:53:26.934322+00:00: called fact_2


120

# Memoization

In [63]:
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-1)

In [64]:
fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(2)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(2)


8

In [71]:
class Fib:
    def __init__(self):
        self.cache = {1: 1, 2: 1}

    def fib(self, n):
        if n not in self.cache:
            print(f'Calculating fib({n})')
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]

In [75]:
f = Fib()
f.fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

In [76]:
f.cache

{1: 1, 2: 1, 3: 2, 4: 3, 5: 5, 6: 8, 7: 13, 8: 21, 9: 34, 10: 55}

In [80]:
def fib():
    cache = {1: 1, 2: 1}
    def inner(n):
        if n not in cache:
            print(f'Calculating fib({n})')
            cache[n] = inner(n-1) + inner(n-2)
        return cache[n]
    return inner

In [81]:
f = fib()
f(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

In [84]:
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [85]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating

55

In [121]:
def memoize(fn):
    cache = {}
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
            
    return inner

In [122]:
@memoize
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [123]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

In [124]:
fib(11)

Calculating fib(11)


89

In [131]:
def fact_(n):
    print(f'Calculating fact_({n})')
    return 1 if n == 1 else n * fact_(n-1)

In [132]:
fact_(3)

Calculating fact_(3)
Calculating fact_(2)
Calculating fact_(1)


6

In [133]:
fact_(4)

Calculating fact_(4)
Calculating fact_(3)
Calculating fact_(2)
Calculating fact_(1)


24

In [134]:
@memoize
def fact_(n):
    print(f'Calculating fact_({n})')
    return 1 if n == 1 else n * fact_(n-1)

In [135]:
fact_(3)

Calculating fact_(3)
Calculating fact_(2)
Calculating fact_(1)


6

In [137]:
fact_(4)

Calculating fact_(4)


24

In [139]:
# results from factorials upto 4 are cached allready !
fact_(6)

720

In [142]:
from functools import lru_cache

@lru_cache
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [143]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

# Decorator Parameters

In [148]:
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            total_elapsed += (perf_counter()-start)
        avg_elapsed = total_elapsed / 10
        print(f'{fn.__name__} took {avg_elapsed}s')
        return result
    
    return inner

In [151]:
@timed
def myfunc():
    return 'hi brk'

In [152]:
myfunc()

myfunc took 3.418070264160633e-07s


'hi brk'

In [208]:
def outer(reps):  # outer is not a decorator it creates a decorator and hence its called decorator factory
    def timed(fn):
        from time import perf_counter
        from functools import wraps

        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                total_elapsed += (perf_counter()-start)
            avg_elapsed = total_elapsed / reps
            print(f'{fn.__name__} took {avg_elapsed*1000:0.6f}ms over {reps} repitions')
            return result
        
        return inner
        
    return timed   # calling outer(n) returns original decorator with reps set to n as free variable

In [209]:
def myfunc():
    return 'hi brk'

In [210]:
myfunc = outer(10)(myfunc)
myfunc()

myfunc took 0.000192ms over 10 repitions


'hi brk'

In [211]:
@outer(10)
def myfunc2():
    return 'hi brk'

In [212]:
myfunc2()

myfunc2 took 0.000429ms over 10 repitions


'hi brk'

# Decorator Class

In [217]:
class Myclass:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, c):
        print(f'called a:{self.a}, b:{self.b}, c:{c}')


In [218]:
obj = Myclass(10, 20)

In [219]:
obj(100)

called a:10, b:20, c:100


In [225]:
class Myclass:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, fn):
        def inner(*args, **kwargs):
            print(f'called a:{self.a}, b:{self.b}')
            return fn(*args, **kwargs)
        return inner

In [229]:
@Myclass(10, 20)
def my_func(s):
    print(f'{s}')

In [230]:
my_func('brk')

called a:10, b:20
brk


In [234]:
def my_func(s):
    print(f'{s}')

In [235]:
obj = Myclass(10, 20)

In [236]:
my_func = obj(my_func)

In [237]:
my_func('hello')

called a:10, b:20
hello


# Decorating Classes

In [238]:
from fractions import Fraction

In [240]:
f = Fraction(2, 3)
f.denominator, f.numerator

(3, 2)

In [241]:
f.speak()

AttributeError: 'Fraction' object has no attribute 'speak'

In [242]:
Fraction.speak = lambda self, message: f'fraction says {message}'

In [243]:
f.speak('i am brk')

'fraction says i am brk'

# this changing of classes or functions from outside is called `monkey patching`

In [244]:
f1 = Fraction(2,3)

In [245]:
f1.is_integal()

AttributeError: 'Fraction' object has no attribute 'is_integal'

In [246]:
Fraction.is_integral = lambda self: self.denominator == 1

In [247]:
f1.is_integral()

False

In [250]:
f2 = Fraction(64, 8)
f2, f2.is_integral()

(Fraction(8, 1), True)

# Lets use decorators for `monkey patching`

In [260]:
from datetime import datetime, timezone

In [269]:
def info(self):
        results=[]
        results.append(f'time: {datetime.now(timezone.utc)}')
        results.append(f'Class: {self.__class__.__name__}')
        results.append(f'id: {hex(id(self))}')
        for k,v in vars(self).items():
            results.append(f'{k}: {v}')
        return results

def debug_info(cls):     
    cls.debug = info
    return cls

In [270]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year

    def say_hi():
        return 'Hello there'

In [271]:
p = Person('Bharath', 1985)

In [272]:
p.debug()

['time: 2025-06-21 21:13:10.492158+00:00',
 'Class: Person',
 'id: 0x107c72cf0',
 'name: Bharath',
 'birth_year: 1985']

# creating static methods using decorators

In [280]:
import math

In [281]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point({self.x}, {self.y})'

    def __abs__(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented
            

In [282]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [283]:
p1 == p2

True

In [284]:
p3 < p2

True

In [285]:
p2 > p3

True

In [286]:
p2 <= p1

TypeError: '<=' not supported between instances of 'Point' and 'Point'

# less than equal can be a combination of less than and equal to
- a <= b iff a < b or a == b
- a > b iff not(a < b) and a != b
- a >= b iff not(a < b)

In [287]:
def complete_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not (self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not (self < other)
    return cls

In [288]:
@complete_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point({self.x}, {self.y})'

    def __abs__(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [289]:
p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)

In [290]:
p2 <= p1

True

In [292]:
p2 != p1

False

In [293]:
from functools import total_ordering

In [294]:
@total_ordering
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point({self.x}, {self.y})'

    def __abs__(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [297]:
p1, p2, p3, p4 = Point(2, 3), Point(2, 3), Point(0, 0), Point(100, 100)

In [296]:
p2 <= p1

True

In [299]:
p4 >= p1

True

In [300]:
p3 <= p1

True